In [3]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp
import trimesh
import open3d as o3d

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return (cmap * 255).astype(np.uint8)

In [6]:
o3d.visualization.webrtc_server.enable_webrtc()


[Open3D INFO] WebRTC GUI backend enabled.


RuntimeError: [1;31m[Open3D Error] (void open3d::visualization::gui::Application::SetWindowSystem(std::shared_ptr<open3d::visualization::gui::WindowSystem>)) /root/Open3D/cpp/open3d/visualization/gui/Application.cpp:344: Cannot set WindowSystem. It is already set.
[0;m

In [ ]:
# Acquiring submesh
mesh_anno = trimesh.load('../gcasp_more/to_use_grasp_transfer/pred/mesh_0.obj')
mesh_anno_verts = mesh_anno.vertices
mesh_anno_faces = mesh_anno.faces

prim_anno = np.load('../gcasp_more/to_use_grasp_transfer/attrs_0.npy')
vert2prim_anno = np.load('../gcasp_more/to_use_grasp_transfer/correspondence_0.npy')
prim_query = 28

triangle_ids = np.arange(len(mesh_anno_faces))
valid_face_ids = []
for i_vert, vert in enumerate(np.array(mesh_anno.vertices)):
    i_prim = vert2prim_anno[i_vert]
    if i_prim == prim_query:
        connected_triangles = np.logical_or(np.logical_or(
                                        mesh_anno_faces[:, 0] == i_vert, 
                                        mesh_anno_faces[:, 1] == i_vert),
                                        mesh_anno_faces[:, 2] == i_vert)
        valid_face_id = triangle_ids[connected_triangles]
        valid_face_ids += valid_face_id.tolist()

        
print(len(valid_face_ids))
sub_mesh_prim = mesh_anno.submesh([valid_face_ids], append=True)
mesh_o3d_prim = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(sub_mesh_prim.vertices), 
                                     o3d.utility.Vector3iVector(sub_mesh_prim.faces))
mesh_o3d = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(mesh_anno.vertices), 
                                     o3d.utility.Vector3iVector(mesh_anno.faces))
colors = np.ones_like(sub_mesh_prim.vertices) * 0.5
colors[:, 0] = 255
mesh_o3d_prim.vertex_colors = o3d.utility.Vector3dVector(colors)
mesh_o3d_prim.compute_vertex_normals()
mesh_o3d.compute_vertex_normals()
o3d.visualization.draw([mesh_o3d_prim, mesh_o3d])

757
[Open3D INFO] Window window_0 created.
[Open3D INFO] EGL headless mode enabled.
[Open3D INFO] ICE servers: ["stun:stun.l.google.com:19302", "turn:user:password@34.69.27.100:3478", "turn:user:password@34.69.27.100:3478?transport=tcp"]
[Open3D INFO] Set WEBRTC_STUN_SERVER environment variable add a customized WebRTC STUN server.
[Open3D INFO] WebRTC Jupyter handshake mode enabled.


In [ ]:
mesh1 = trimesh.load('../gcasp_more/to_use_grasp_transfer/pred/mesh_0.obj')
mesh2 = trimesh.load('../gcasp_more/to_use_grasp_transfer/pred/mesh_2.obj')
mesh1 = TriMesh(mesh1.vertices, mesh1.faces)
mesh2 = TriMesh(mesh2.vertices, mesh2.faces)
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')
# double_plot(mesh1,mesh2)

In [ ]:
process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
    'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5],  # loading 5 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
}

model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);

In [ ]:
fit_params = {
    'w_descr': 1e0,
    'w_lap': 1e-2,
    'w_dcomm': 1e-1,
    'w_orient': 0
}



model.fit(**fit_params, verbose=True)

In [ ]:
p2p_21 = model.get_p2p(n_jobs=1)
# Mesh 1 : 7207 vertices, 14410 faces
# Mesh 2 : 5000 vertices,  9996 faces

cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
# double_plot(mesh1,mesh2,cmap1,cmap2)

m1 = trimesh.Trimesh(mesh1.vertlist, mesh1.facelist)
m2 = trimesh.Trimesh(mesh2.vertlist, mesh2.facelist)

m1.visual.vertex_colors = cmap1
m2.visual.vertex_colors = cmap2
corr = []
i = 0
for i2, v2 in enumerate(m2.vertices):
    if i2 % 5000 == 0:
        i1 = p2p_21[i2]
        v1 = m1.vertices[i1]
        line = np.concatenate((v1[None], v2[None]+np.array([2,0,0])[None]),axis=0)
        corr.append(line[None]) 
corr = np.concatenate(corr, axis=0)
corr_vis = trimesh.load_path(corr)
colors = np.array([255, 0, 0, 255]).astype(np.uint8)[None].repeat(len(corr_vis.entities), axis=0)
print(colors.shape, len(corr_vis.entities))
corr_vis.colors = colors
trimesh.Scene([m1, m2.apply_translation([2,0,0]), corr_vis]).show()
